In [4]:
import numpy as np
import cvxpy as cp

In [5]:
gains = np.array([[1.0,0.1,0.2,0.1,0.0], [0.1,1.0,0.1,0.1,0.0], [0.2,0.1,2.0,0.2,0.2], [0.1,0.1,0.2,1.0,0.1], [0.0,0.0,0.2,0.1,1.0]])

n,m = np.shape(gains)
max_Power = np.array([3.]*n).reshape((n,1))
rx_Power = (np.array([5.,5.,5.,5.,5.]).reshape((n,1)))/n
sigma = np.array([0.5,0.5,0.5,0.5,0.5]).reshape((n,1))
grps = np.array([[1.,1.,0,0,0],[0,0,1.,1.,1.]])
max_grp = np.array([[3.0],[6.0]])
epsilon = 0.05

p = cp.Variable((n,1))
optimalPow = np.zeros(n)

# upper and lower bounds
upper = 1e4
lower = 0
  
alpha = cp.Parameter()
constraints = [(gains-gains*np.eye(n))@p + sigma <= alpha*(gains*np.eye(n))*p, 
               p <= max_Power , p >= 0, gains*p <= rx_Power, np.array([group/np.sum(group) for group in grps])*p <= max_grp]

obj = cp.Minimize(alpha)
alpha.value = upper
prob = cp.Problem(obj, constraints)
prob.solve()

if prob.status != 'optimal':
  print('Optimal solution out of bounds\n')
  alpha.value = np.nan
  optimalPow = np.nan

else:  
  alpha.value = lower
  prob = cp.Problem(obj, constraints)
  prob.solve()

  if prob.status == 'optimal':
    print('Optimal solution out of bounds\n')
    alpha.value = np.nan
    optimalPow = np.nan

  else:
    # Bisection algortithm 
    maxLoop = int(1e7)
    for i in range(1,maxLoop):
      alpha.value = (upper + lower)/2.0
    
      if upper-lower <= epsilon:
        break
      prob = cp.Problem(obj, constraints)
      prob.solve()
      if prob.status == 'optimal':
        upper = alpha.value
        optimalPow = p.value
      else:
        lower = alpha.value
      if upper - lower > epsilon and i == (maxLoop-1):
        print("Solution not converged to order epsilon")

print('Minimum SINR=%.4g'%(1/alpha.value))
print('Power=%s'%(optimalPow))

Minimum SINR=0.7598
Power=[[0.51316491]
 [0.49975582]
 [0.32181663]
 [0.54193416]
 [0.46441846]]
